In [1]:
import pandas as pd
df = pd.read_csv('../data/epi-recipes.csv')
df.shape

(35441, 25)

In [2]:
df['healthy'] = [1 if 'Healthy' in t else 0 for t in df.tags]
df["RRR"] = ((df['nutritions.Protein']/50 + df['nutritions.Fiber']/25)/2) / ((df['nutritions.Calories']/2000 + df['nutritions.Saturated Fat']/20 + df['nutritions.Cholesterol']/300 + df['nutritions.Carbohydrates']/300 + df['nutritions.Sodium']/2400)/5)
df.shape

(35441, 27)

In [3]:
df['activetime'] = df['activetime'].fillna('')
df['desc'] = df['desc'].fillna('')
df['imgurl'] = df['imgurl'].fillna('')
df['ingredients'] = df['ingredients'].fillna('')
df['reviews'] = df['reviews'].fillna('')
df['steps'] = df['steps'].fillna('')
df['tags'] = df['tags'].fillna('')
df['title'] = df['title'].fillna('')
df['totaltime'] = df['totaltime'].fillna('')
df['url'] = df['url'].fillna('')
df['yiel'] = df['yiel'].fillna('')
df['makeagain'] = df['makeagain'].fillna(0)
df['rating'] = df['rating'].fillna(0)
df['reviewcount'] = df['reviewcount'].fillna(0)
df['RRR'] = df['RRR'].fillna(0)
df['serving'] = df['serving'].fillna(0)
df['nutritions.Calories'] = df['nutritions.Calories'].fillna(0)
df['nutritions.Carbohydrates'] = df['nutritions.Carbohydrates'].fillna(0)
df['nutritions.Cholesterol'] = df['nutritions.Cholesterol'].fillna(0)
df['nutritions.Fat'] = df['nutritions.Fat'].fillna(0)
df['nutritions.Fiber'] = df['nutritions.Fiber'].fillna(0)
df['nutritions.Monounsaturated Fat'] = df['nutritions.Monounsaturated Fat'].fillna(0)
df['nutritions.Polyunsaturated Fat'] = df['nutritions.Polyunsaturated Fat'].fillna(0)
df['nutritions.Protein'] = df['nutritions.Protein'].fillna(0)
df['nutritions.Saturated Fat'] = df['nutritions.Saturated Fat'].fillna(0)
df['nutritions.Sodium'] = df['nutritions.Sodium'].fillna(0)

In [5]:
df.to_csv('../data/for_search.csv')

In [4]:
from whoosh.qparser import *
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED,NUMERIC
from whoosh.analysis import StemmingAnalyzer,StandardAnalyzer
from whoosh import index
import os, os.path

In [5]:
schema = Schema(title = TEXT(stored=True),
                desc = TEXT(stored=True),
                ingredients = TEXT(stored=True),
                steps = TEXT(stored=True),
                reviews = TEXT(stored=True),
                tags = TEXT(stored=True))

In [6]:
def write(data):
    writer.add_document(title = data['title'],
                        desc = data['desc'],
                        ingredients = data['ingredients'],
                        steps = data['steps'],
                        reviews = data['reviews'],
                        tags = data['tags'])

In [14]:
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = index.create_in("indexdir", schema)
ix = index.open_dir("indexdir")
writer = ix.writer()

In [15]:
df1 = df[:1000]
for idx, row in df1.iterrows():
    write(row)

In [16]:
writer.commit()

In [17]:
writer = ix.writer()
df2 = df[1000:5000]
for idx, row in df2.iterrows():
    write(row)

In [18]:
writer.commit()

In [19]:
writer = ix.writer()
df3 = df[5000:8000]
for idx, row in df3.iterrows():
    write(row)

In [20]:
writer.commit()

In [21]:
writer = ix.writer()
df4 = df[8000:11000]
for idx, row in df4.iterrows():
    write(row)

In [22]:
writer.commit()

In [23]:
writer = ix.writer()
df5 = df[11000:15000]
for idx, row in df5.iterrows():
    write(row)

In [24]:
writer.commit()

In [25]:
writer = ix.writer()
df6 = df[15000:18000]
for idx, row in df6.iterrows():
    write(row)

In [26]:
writer.commit()

In [27]:
writer = ix.writer()
df7 = df[18000:21000]
for idx, row in df7.iterrows():
    write(row)

In [28]:
writer.commit()

In [29]:
writer = ix.writer()
df8 = df[21000:25000]
for idx, row in df8.iterrows():
    write(row)

In [30]:
writer.commit()

In [31]:
writer = ix.writer()
df9 = df[25000:28000]
for idx, row in df9.iterrows():
    write(row)

In [32]:
writer.commit()

In [33]:
writer = ix.writer()
df10 = df[28000:31000]
for idx, row in df10.iterrows():
    write(row)

In [34]:
writer.commit()

In [35]:
writer = ix.writer()
df18 = df[31000:35441]
for idx, row in df18.iterrows():
    write(row)

In [36]:
writer.commit()

In [43]:
parser = QueryParser("ingredients",schema=schema) 
result = parser.parse(u"chicken")
print(result)

ingredients:chicken


In [85]:
titles = []
with ix.searcher() as searcher:
    results = searcher.search(result,limit=20)
    for r in results:
        titles.append(r['title'])

In [86]:
len(titles)

20

In [87]:
type(df[df['title']==titles[0]])

pandas.core.frame.DataFrame

In [97]:
result_list = pd.DataFrame(columns=df.columns.copy())
result_list.shape

(0, 27)

In [118]:
for t in titles:
    result_list = result_list.append(df[df['title']==t])
    if df[df['title']==t].shape[0] > 1:
        print(df[df['title']==t].title)

21875    Parmesan Polenta
24094    Parmesan Polenta
Name: title, dtype: object
19393    Creamy Grits
34734    Creamy Grits
Name: title, dtype: object
28662    Chicken Long Rice
34436    Chicken Long Rice
Name: title, dtype: object


In [119]:
result_list = result_list.drop_duplicates()
result_list.shape

(23, 27)

In [120]:
r = result_list.sort_values(by=['RRR'],ascending=False)

In [121]:
r.shape

(23, 27)

In [123]:
r[['title','RRR','healthy']]

,title,RRR,healthy
10948,Grilled Bone-in Chicken Breasts and Pieces,1.830198,1
8258,Dr. Lee's Red Wine Chicken Stew,1.670895,1
196,Lentil and Chicken Soup with Sweet Potatoes an...,1.551457,1
35197,Chicken Soup,1.520027,0
28586,"Chicken with Prosciutto, Rosemary, and White Wine",1.384083,0
29530,Sage-Roasted Chicken with Madeira Sauce,1.292965,0
13968,Chicken Kishk,1.230583,0
19308,Maple-Barbecued Chicken,1.191281,0
27663,Grilled Chicken Moroccan Style,1.094807,0
34436,Chicken Long Rice,1.011911,0
